In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv('C:/Users/user/Desktop/Data Science or Data Analytics/Github/Data-Analytics-on-Chips-Category/cleaned_dataset.csv')

In [4]:
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE(g),BRAND_NAME,LIFESTAGE,PREMIUM_CUSTOMER,Month
0,2018-07-03,19,19205,16466,26,Pringles Sweet&Spcy BBQ 134g,1,3.7,134,Pringles,OLDER SINGLES/COUPLES,Mainstream,2018-07
1,2018-07-03,189,189381,190189,84,GrnWves Plus Btroot & Chilli Jam 180g,1,3.1,180,Grain,OLDER FAMILIES,Mainstream,2018-07
2,2018-07-03,124,124236,127984,104,Infuzions Thai SweetChili PotatoMix 110g,1,3.8,110,Infuzions,OLDER FAMILIES,Budget,2018-07
3,2018-07-03,70,70131,68241,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,Kettle,RETIREES,Premium,2018-07
4,2018-07-03,33,33140,30342,10,RRD SR Slow Rst Pork Belly 150g,2,5.4,150,RRD,YOUNG FAMILIES,Mainstream,2018-07


In [5]:
# Change the dtype for date to datetime format
df['DATE'] = pd.to_datetime(df['DATE'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DATE              264836 non-null  datetime64[ns]
 1   STORE_NBR         264836 non-null  int64         
 2   LYLTY_CARD_NBR    264836 non-null  int64         
 3   TXN_ID            264836 non-null  int64         
 4   PROD_NBR          264836 non-null  int64         
 5   PROD_NAME         264836 non-null  object        
 6   PROD_QTY          264836 non-null  int64         
 7   TOT_SALES         264836 non-null  float64       
 8   PACK_SIZE(g)      264836 non-null  int64         
 9   BRAND_NAME        264836 non-null  object        
 10  LIFESTAGE         264836 non-null  object        
 11  PREMIUM_CUSTOMER  264836 non-null  object        
 12  Month             264836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(6), object(5)
memor

# GETTING THE METRICS THAT'LL BE USED TO FIND THE CONTROL STORES

METRICS TO FIND:
* total sales revenue* total number of customers
* Average number of transactions per customer

Consider the monthly sales experience of eachcustomer


In [8]:
def create_metrics(df):

    # Grouping by STORE_NBR and month (M) from the DATE column
    df['Month'] = df['DATE'].dt.to_period('M')
    
    # Total sales revenue
    T_sales = df.groupby([ 'Month','STORE_NBR'])['TOT_SALES'].sum().reset_index()

    # Total number of customers
    T_Cust = df.groupby(['Month', 'STORE_NBR'])['LYLTY_CARD_NBR'].nunique().reset_index()

    # Average number of transactions per customer
    Avg_Tcust =  df.groupby(['Month', 'STORE_NBR'])['LYLTY_CARD_NBR'].count() / \
                df.groupby(['Month', 'STORE_NBR'])['LYLTY_CARD_NBR'].nunique()
    #print(Avg_Tcust)
    
   # Merge the metrics while considering the months
    result1 = pd.merge(T_sales, T_Cust, how='outer', on=[ 'Month', 'STORE_NBR'])
    #print(result1.columns)
    result2 =  pd.merge(result1, Avg_Tcust, how='outer', on=['Month','STORE_NBR'])
    #print(result2.columns)  # Check the actual number of columns

   # Clarifying the name of the columns;
    result2.columns = [ 'Month', 'STORE_NBR', 'M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']
    
    return result2

Metrics = create_metrics(df)
metrics = pd.DataFrame(Metrics)
metrics


,Month,STORE_NBR,M_SalesRevenue,M_NumberOfCustomers,M_AvgTranscPerCustomers
0,2018-07,1,183.4,46,1.043478
1,2018-07,2,141.2,36,1.055556
2,2018-07,3,1140.1,106,1.235849
3,2018-07,4,1326.5,122,1.245902
4,2018-07,5,776.2,90,1.277778
...,...,...,...,...,...
3411,2019-07,268,10.4,2,1.000000
3412,2019-07,269,65.2,8,1.000000
3413,2019-07,270,32.2,4,1.000000
3414,2019-07,271,76.4,11,1.000000


# SELECT CONTROL STORES
#### USING THE METRICS TO FIND THE CONTROL STORES FOR THE GIVEN TRIAL STORES

Trial Stores: 77, 86 and 88
* Write a function* Consider using Pearson correlatio
*  Or metric such as a magnitude distance e.g Observed distance - minimum distance)/(Maximum distance - minimum distance) as a measre.

My ideas:

* Speicify the trial stores in the function.
* Create another dataframe with just the trial stores and their respective metrics present
* Create another dataframe that excludes the trial stores within the function.
* Use Pearson correlation to measure the relationship between the trial stores metrics and the other stores metrics
* * Aggregate metrics by Month i.e calculate the monthly average of your three metrics for both groups
  * Merge the dataframes for comparison
  * Compute Pearson correlation between the trial stores and other stores for each metrics.

In [11]:
# Defining the trial stores


def find_controlstores(metrics, corr_min= 0.85, corr_max=0.95):
    
    trial_stores = [77, 86, 88]   # Defined trial stores
    control_stores = {} # Dictionary to store selected control stores
   
    trial_df = metrics[metrics['STORE_NBR'].isin(trial_stores)]
    
    other_stores = metrics[~metrics['STORE_NBR'].isin(trial_stores)]
    
    
    # Aggregate metrics by Month
    trial_metrics = trial_df.groupby(['Month', 'STORE_NBR'])[['M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']].mean()
    #print(trial_metrics)
    
    # Aggregate metrics for other stores
    other_metrics = other_stores.groupby(['Month', 'STORE_NBR'])[['M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']].mean()
    #print(other_metrics)

    # Pivot the data to have stores as rows and metrics over time as columns
    trial_pivot = trial_metrics.unstack(level=1)
    #print(trial_pivot)
    other_pivot = other_metrics.unstack(level=1)
    #print(other_pivot)

    for trial_store in trial_stores:
        print(f"\n### Correlation for Trial Store {trial_store} ###")

        # Extract data for the current trial store
        trial_store_metrics = trial_pivot.xs(trial_store, axis=1, level=1) # Return cross-section from the Series/DataFrame.
        #print(trial_store_metrics)
        # Compute correlation with all other stores
        ''' .corrwith() is a method that computes pairwise correlation between rows or columns of two dataframes or a dataframe and a series.
                    A fast way to compare how closely related the data aross the two datasets is, based on Pearson correlation by default.
                    '''
        correlations = other_pivot.corrwith(trial_store_metrics, axis=1)  
        print(correlations)
        # Convert to a dataframe for better readavitlity
        corr_df = correlations.reset_index()
        corr_df.columns = ['STORE_NBR', 'Correlation']

        # Filter stores within the given correlation range
        suitable_stores = corr_df[(corr_df['Correlation'] >= corr_min) & (corr_df['Correlation'] <= corr_max)]

        # Store selected control stores for this trial store
        control_stores[trial_store] = suitable_stores['STORE_NBR'].tolist()

        # Display correlations
        print(suitable_stores)

    return control_stores  # Returns a dictionary of selected control stores
                                   

    '''
    # Merge trial and other stores' metrics on Month
    merged_metrics = trial_metrics.merge(other_metrics, on="Month", suffixes=("_trial", "_other"))

    # Compute Pearson correlation for each metric
    correlations = merged_metrics.corr(method='pearson')

    # Display correlations between trial and other stores
    print(correlations.loc[['M_SalesRevenue_trial', 'M_NumberOfCustomers_trial', 'M_AvgTranscPerCustomers_trial'],
          ['M_SalesRevenue_other', 'M_NumberOfCustomers_other', 'M_AvgTranscPerCustomers_other']])

    #return correlations
    '''
print('\n### Final Selected Control Stores ###')
print(find_controlstores(metrics, corr_min= 0.85, corr_max=0.95))    



### Final Selected Control Stores ###

### Correlation for Trial Store 77 ###


ValueError: cannot join with no overlapping index names

In [ ]:
sns.regplot ( x = 'STORE_NBR', y = 'M_SalesRevenue', data = metrics)
plt.ylim(0,)

In [ ]:
def find_controlstores(metrics, trial_stores=[77, 86, 88], corr_min=0.85, corr_max=0.95):
    selected_controls = {}
    metrics_list = ['M_SalesRevenue', 'M_NumCustomers', 'M_AvgTranscPerCustomers']

    # Create a multi-index pivot for all stores
    pivot = metrics.pivot(index="Month", columns="STORE_NBR", values=metrics_list)

    for trial_store in trial_stores:
        print(f'\n>>> Trial Store: {trial_store}')
        trial_corrs = []

        # Loop through all other stores
        for store in pivot.columns.levels[1]:
            if store == trial_store:
                continue  # skip comparing the store with itself

            correlations = []
            for metric in metrics_list:
                trial_series = pivot[(metric, trial_store)]
                other_series = pivot[(metric, store)]

                corr = trial_series.corr(other_series)
                correlations.append(corr)

            avg_corr = np.mean(correlations)

            # Save if correlation is within range
            if corr_min <= avg_corr <= corr_max:
                trial_corrs.append((store, avg_corr))

        # Sort by highest average correlation and take the top match
        if trial_corrs:
            best_match = sorted(trial_corrs, key=lambda x: x[1], reverse=True)[0]
            selected_controls[trial_store] = best_match[0]
            print(f"✓ Selected Control Store: {best_match[0]} with avg correlation: {best_match[1]:.3f}")
        else:
            print(" No control store found within the specified correlation range.")

    return selected_controls


In [ ]:
control_matches = find_controlstores(metrics, corr_min=0.85, corr_max=0.95)
print("\n### Final Selected Control Stores ###")
print(control_matches)


In [12]:

def find_controlstores(metrics, corr_min=0.85, corr_max=0.95):
    trial_stores = [77, 86, 88]
    control_stores = {}
    metrics_list = ['M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']

    for trial_store in trial_stores:
        print(f"\n### Correlation for Trial Store {trial_store} ###")
        trial_df = metrics[metrics['STORE_NBR'] == trial_store]

        store_corrs = []

        for store in metrics['STORE_NBR'].unique():
            if store == trial_store or store in trial_stores:
                continue  # skip self and other trial stores

            other_df = metrics[metrics['STORE_NBR'] == store]

            avg_corrs = []
            for metric in metrics_list:
                merged = trial_df[['Month', metric]].merge(
                    other_df[['Month', metric]],
                    on='Month',
                    suffixes=('_trial', '_other')
                )

                if len(merged) >= 2:  # Ensure there's enough overlap to compute correlation
                    corr = merged[f'{metric}_trial'].corr(merged[f'{metric}_other'])
                    avg_corrs.append(corr)

            # Compute average correlation for this store
            if len(avg_corrs) == len(metrics_list):  # only keep if all metrics have valid correlations
                mean_corr = np.mean(avg_corrs)
                if corr_min <= mean_corr <= corr_max:
                    store_corrs.append((store, mean_corr))

        # Pick the store with the highest correlation
        if store_corrs:
            best_control = sorted(store_corrs, key=lambda x: x[1], reverse=True)[0]
            control_stores[trial_store] = best_control[0]
            print(f"✓ Selected Control Store: {best_control[0]} with avg correlation: {best_control[1]:.3f}")
        else:
            print(" No suitable control store found for this trial store.")

    return control_stores

# Usage
print('\n### Final Selected Control Stores ###')
print(find_controlstores(metrics, corr_min=0.85, corr_max=0.95))



### Final Selected Control Stores ###

### Correlation for Trial Store 77 ###


C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base

 No suitable control store found for this trial store.

### Correlation for Trial Store 86 ###


C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base

✓ Selected Control Store: 156 with avg correlation: 0.917

### Correlation for Trial Store 88 ###


C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base

✓ Selected Control Store: 230 with avg correlation: 0.947
{86: 156, 88: 230}


C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3-2024\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


## RESULT:

* Trial Store 88: Control Store 156 (Avg. correlation: 0.917)
* Trial Store 86: Control Store 230 (Avg. correlation: 0.947)
* Trial Store 77: No suitable control store found.

Trial Stores 86 and 88 have valid control stores with strong correlations (above 0.9), which means you can confidently continue to assess the trial's impact on the stores.

Unfortunatly, trial store 77 has no suitable control store due to low similarity; so no impact assessment will proceed for this particular store.

## IMPACT ASSESSMENT STAGE

Trial stores 86 and 88 will be compared with their matched control stores (230 and 156) during the trial period (July 2019 to June 2020)

#### Things to look out for:
* Check if the change in total sales during the trial period is statisticallty significant.
* Is the lift due to an increase in the number of customers?
* Or an increase in acerage number of transactions per customer?
* Or both?

In [29]:
# If Month column is a Period type, convert it to Timestamp
if pd.api.types.isinstance_dt(metrics['Month']):
    metrics['Month'] = metrics['Month'].dt.to_timestamp()


C:\Users\user\AppData\Local\Temp\ipykernel_14324\918735948.py:2: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(metrics['Month']):


In [35]:
# Filter Trial Period Data

trial_start = '2019-07'
trial_end = '2020-06'

# Convert Month to datetime in case
metrics['Month'] = pd.to_datetime(metrics['Month'])

# Filter datra for trial period
trial_period_data = metrics[
    (metrics['Month'] >= trial_start) &
    (metrics['Month'] <= trial_end)
]

In [37]:
# Define Trial-Control Pairs

trial_control_pairs = {
    86: 230,
    88: 156
}

In [43]:
# Loop Through Pairs to Compare

import scipy.stats as stats

for trial, control in trial_control_pairs.items():
    trial_data = trial_period_data[trial_period_data['STORE_NBR'] == trial]
    control_data = trial_period_data[trial_period_data['STORE_NBR'] == control]

    print(f"\n--- Trial Store {trial} vs Control Store {control} ---")

    # Total Sales
    trial_sales = trial_data['M_SalesRevenue']
    control_sales = control_data['M_SalesRevenue']

    # T-test for sales difference
    t_stat, p_val = stats.ttest_ind(trial_sales, control_sales, equal_var=False)
    if p_val < 0.05:
        print(" Significant difference in total sales during trial period.")

        # For the others:
        trial_cust = trial_data['Monthly_Customers'].mean()
        control_cust = control_data['Monthly_Customers'].mean()

        trial_trans = trial_data['Avg_Transactions_Per_Customer'].mean()
        control_trans = control_data['Avg_Transactions_Per_Customer'].mean()

        print(f"Avg Monthly Customers - Trial: {trial_cust:.2f}, control: {control_cust:.2f}")
        print(f"Avg Transactions/Customer - Trial: {trial_trans:.2f}, control: {control_trans:.2f}")

        cust_change = ((trial_cust - control_cust) / control_cust) * 100
        trans_change = ((trial_trans - control_trans) / control_trans) * 100

        print(f"Customer Increas; {cust_change:.2f}%")
        print(f"Transactions/Customer Increase: {trans_change:.2f}%")

    else:
        print("No significant difference in total sales during the trial period.")


--- Trial Store 86 vs Control Store 230 ---
No significant difference in total sales during the trial period.

--- Trial Store 88 vs Control Store 156 ---
No significant difference in total sales during the trial period.


C:\Users\user\anaconda3-2024\Lib\site-packages\scipy\stats\_stats_py.py:1087: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\user\anaconda3-2024\Lib\site-packages\scipy\stats\_stats_py.py:1087: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
